## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,77.00,78,90,10.36,light rain
1,1,Prince Rupert,CA,54.3161,-130.3201,50.00,71,20,4.61,few clouds
2,2,Atuona,PF,-9.8000,-139.0333,80.42,78,96,14.32,overcast clouds
3,3,Qaanaaq,GL,77.4840,-69.3632,25.14,61,97,5.93,overcast clouds
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,71.71,75,75,6.13,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 50
What is the maximum temperature you would like for your vacation? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,77.00,78,90,10.36,light rain
1,1,Prince Rupert,CA,54.3161,-130.3201,50.00,71,20,4.61,few clouds
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,71.71,75,75,6.13,broken clouds
5,5,Cape Town,ZA,-33.9258,18.4232,64.00,66,0,3.44,clear sky
6,6,Saint-Philippe,RE,-21.3585,55.7679,75.20,69,0,6.91,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,77.00,light rain,-21.2078,-159.7750,
1,Prince Rupert,CA,50.00,few clouds,54.3161,-130.3201,
4,Arraial Do Cabo,BR,71.71,broken clouds,-22.9661,-42.0278,
5,Cape Town,ZA,64.00,clear sky,-33.9258,18.4232,
6,Saint-Philippe,RE,75.20,clear sky,-21.3585,55.7679,
7,Katsuura,JP,69.01,few clouds,35.1333,140.3000,
8,Vares,BA,51.80,broken clouds,44.1644,18.3283,
9,Rikitea,PF,77.79,scattered clouds,-23.1203,-134.9692,
11,Lebu,CL,52.72,overcast clouds,-37.6167,-73.6500,
12,Poum,NC,78.03,few clouds,-20.2333,164.0167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel found")
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found... skipping.
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found... skipping.
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found... skipping.
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found... skippi

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))